## **Santander Dev Week 2025 (ETL com Python)**

- **Contexto**: Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Cientista de Dados**: Lucca de Sena Barbosa

In [2]:
import pandas as pd

### 🟡 **Opção 2: Mais Completa (Leitura de Arquivo)**

#### **Extract**

In [4]:
users = pd.read_csv('clientes.csv').to_dict(orient='records')
users

[{'ID': 1,
  'Nome': 'Lucas Pereira',
  'Conta': '00012345-6',
  'Cartão': 4539876543211023},
 {'ID': 2,
  'Nome': 'Marina Alves',
  'Conta': '00023456-7',
  'Cartão': 5198347623459081},
 {'ID': 3,
  'Nome': 'Rafael Costa',
  'Conta': '00034567-8',
  'Cartão': 4029384756102837},
 {'ID': 4,
  'Nome': 'Ana Beatriz',
  'Conta': '00045678-9',
  'Cartão': 4916728301947562},
 {'ID': 5,
  'Nome': 'Bruno Silva',
  'Conta': '00056789-0',
  'Cartão': 5274019384756201},
 {'ID': 6,
  'Nome': 'Carla Mendes',
  'Conta': '00067890-1',
  'Cartão': 4483927561029384},
 {'ID': 7,
  'Nome': 'Diego Rocha',
  'Conta': '00078901-2',
  'Cartão': 6011459837204568},
 {'ID': 8,
  'Nome': 'Fernanda Lima',
  'Conta': '00089012-3',
  'Cartão': 5321984762039457},
 {'ID': 9,
  'Nome': 'Gustavo Nogueira',
  'Conta': '00090123-4',
  'Cartão': 4556729301847560},
 {'ID': 10,
  'Nome': 'Helena Martins',
  'Conta': '00101234-5',
  'Cartão': 5109384726159083}]

#### **Transform e Load**

- Implementei a API do Groq utilizando o modelo ``llama-4-maverick-17b-128e-instruct`` para a geração de mensagens de marketing personalizadas. A escolha do modelo se dar por ser uma tecnologia de 2025 baseada na arquitetura MoE (128 especialistas), o que garante superioridade em lógica e precisão comparado a versões legadas. Além disso, a infraestrutura da Groq foi selecionada por oferecer acesso gratuito a modelos de alta performance, proporcionando uma alternativa economicamente viável à API da OpenAI.

In [ ]:
from groq import Groq
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq()

for user in users:

    completion = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
        {
            "role": "user",
            "content": f"""Gere uma mensagem de marketing personalizada para o cliente com base nas suas preferências e histórico de compras. Essa mensagem deve enfatizar a importância dos investimentos. Utilize uma linguagem persuasiva e amigável, destacando os benefícios dos produtos financeiros oferecidos pelo banco Santander. A mensagem deve ser clara, concisa e adaptada ao perfil do cliente. Aqui estão os detalhes do cliente:
            {user}"""
        }
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None
    )

    new = ""

    for chunk in completion:
        new += chunk.choices[0].delta.content or ""

    # A etapa de Load foi feita aqui, uma vez que não foi utilizado uma API externa para armazenar os dados.

    user['news'].append(new)   

In [23]:
users

[{'ID': 1,
  'Nome': 'Lucas Pereira',
  'Conta': '00012345-6',
  'Cartão': 4539876543211023,
  'news': ['Olá, Lucas Pereira!\n\nEsperamos que esteja tudo bem! Como cliente Santander, você faz parte de uma comunidade que valoriza a segurança, a inovação e o compromisso com o seu sucesso financeiro. Queremos agradecer por confiar nos nossos serviços e produtos. Seu histórico conosco nos permite entender melhor suas necessidades e oferecer soluções personalizadas que atendam às suas expectativas.\n\n**Seu Perfil Financeiro e Nossas Recomendações**\n\nEmbora não tenhamos registros de compras ou interações recentes, entendemos que planejar para o futuro é uma prioridade para muitos de nossos clientes. É por isso que gostaríamos de compartilhar algumas informações valiosas sobre como você pode começar ou continuar a construir uma estratégia de investimentos sólida.\n\n### Por que investir é importante?\n\nInvestir não é apenas uma forma de aumentar seu patrimônio; é também uma maneira de alc

#### **Mensagens de Marketing Geradas**:

In [27]:
for user in users:
    print(user['news'][-1])
    print(50*"-")

Olá, Lucas Pereira!

Esperamos que esteja tudo bem! Como cliente Santander, você faz parte de uma comunidade que valoriza a segurança, a inovação e o compromisso com o seu sucesso financeiro. Queremos agradecer por confiar nos nossos serviços e produtos. Seu histórico conosco nos permite entender melhor suas necessidades e oferecer soluções personalizadas que atendam às suas expectativas.

**Seu Perfil Financeiro e Nossas Recomendações**

Embora não tenhamos registros de compras ou interações recentes, entendemos que planejar para o futuro é uma prioridade para muitos de nossos clientes. É por isso que gostaríamos de compartilhar algumas informações valiosas sobre como você pode começar ou continuar a construir uma estratégia de investimentos sólida.

### Por que investir é importante?

Investir não é apenas uma forma de aumentar seu patrimônio; é também uma maneira de alcançar metas importantes, como comprar uma casa, financiar a educação dos filhos ou garantir uma aposentadoria tranq